In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import glob
import math
import librosa
import itertools
import numpy as np
import sklearn.metrics as metrics

import tensorflow as tf
from keras.models import Sequential
from keras.initializers import Constant, TruncatedNormal
from keras.layers import Dense, Activation, Flatten
from keras.layers.convolutional import Convolution2D, Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, Callback

Using TensorFlow backend.


In [2]:
train_features, train_labels = np.load('train_features.npy'), np.load('train_labels.npy')
test_features, test_labels = np.load('test_features.npy'), np.load('test_labels.npy')
num_classes = 4
cost_history = np.empty(shape=[1],dtype=float)

In [3]:
class EvaluateValidation(Callback):      
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        global cost_history
        cost_history = np.append(cost_history,loss)
        print('\nValidation loss: {}, acc: {}, Metric Names: {}\n'.format(loss, acc, self.model.metrics_names))

In [4]:
xxtrain = np.concatenate((np.reshape(train_features[:,:,:,0], (train_features.shape[0], 60 * 41)), np.reshape(train_features[:,:,:,1], (train_features.shape[0], 60 * 41))), axis=1)
xxval = np.concatenate((np.reshape(test_features[:,:,:,0], (test_features.shape[0], 60 * 41)), np.reshape(test_features[:,:,:,1], (test_features.shape[0], 60 * 41))), axis=1)

In [5]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=1e5)
logreg.fit(xxtrain, np.argmax(train_labels, axis=1))

LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [6]:
y_pred = logreg.predict(xxval)

In [7]:
accuracy = 100.0 * np.sum(y_pred == np.argmax(test_labels, axis=1)) / y_pred.shape[0]

In [8]:
print accuracy
print np.sum( np.argmax(test_labels, axis=1) == 0)
print np.sum( np.argmax(test_labels, axis=1) == 1)
print np.sum( np.argmax(test_labels, axis=1) == 2)
print np.sum( np.argmax(test_labels, axis=1) == 3)

print 100.0 * np.sum( np.argmax(test_labels, axis=1) == 0) / test_labels.shape[0]

35.8490566038
85
41
65
21
40.0943396226


In [ ]:
def build_model():
    k_init_1 = Constant(value=1)
    k_init = 'he_normal'
    bias_init = Constant(value=0.1)
    adam = Adam(lr=1e-3, decay=0.75)
#     sgd = SGD(lr=1e-3, momentum=0.5, decay=0.75,  nesterov=True)
    
    model = Sequential()
    model.add(Convolution2D(filters=256, kernel_size=(30, 30), strides=(1,1), padding='same',
                            data_format='channels_last', kernel_initializer=k_init_1, bias_initializer=bias_init,
                            input_shape=(60, 41, 2)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='valid'))
    
    model.add(Flatten())
    model.add(Dense(500, kernel_initializer=k_init, bias_initializer=bias_init))
    
    model.add(Dense(num_classes, kernel_initializer=k_init, bias_initializer=bias_init))
    model.add(Activation('softmax'))
    
    model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = build_model()
model.summary()

from keras.backend import get_session
get_session().run(tf.global_variables_initializer())
a = model.fit(train_features, train_labels, batch_size=50, epochs=200, verbose=1, callbacks=[EvaluateValidation((test_features, test_labels))])

loss_and_metrics = model.evaluate(test_features, test_labels, batch_size=50, verbose=1)
y_pred = model.predict(test_features, batch_size=50, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 41, 256)       461056    
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 41, 256)       1024      
_________________________________________________________________
activation_1 (Activation)    (None, 60, 41, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 20, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 153600)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               76800500  
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 2004      
__________

In [ ]:
print "Loss and accuracy: ", loss_and_metrics

In [ ]:
fig = plt.figure(figsize=(15,10))
plt.plot(cost_history)
# plt.axis([0,training_epochs,0,np.max(cost_history)])
plt.show()

In [ ]:
y_test_class = np.argmax(test_labels, axis=1)
y_pred_class = np.argmax(y_pred, axis=1)
conf_mat = metrics.confusion_matrix(y_test_class, y_pred_class)

In [ ]:
# Compute ROC curve and ROC area for each class
n_classes = 4
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(n_classes):
    fpr[i], tpr[i], _ = metrics.roc_curve(test_labels[:, i], y_pred[:, i])
    roc_auc[i] = metrics.auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = metrics.roc_curve(test_labels.ravel(), y_pred.ravel())
roc_auc["micro"] = metrics.auc(fpr["micro"], tpr["micro"])

plt.figure()
lw = 2
plt.plot(fpr["micro"], tpr["micro"], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc["micro"])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.show()

roc_score = metrics.roc_auc_score(test_labels, y_pred)

#print "Loss: {} Accuracy: {}%".format(loss_and_metrics[0],loss_and_metrics[1] * 100)
print "ROC AUC Score: ", roc_score

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
plot_confusion_matrix(conf_mat, classes=['BD15C', 'BD6B', 'BD6A', 'BD15A'])